In [1]:
pip install openai==0.28

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests, os, random, time, csv, json
from collections import namedtuple
from bs4 import BeautifulSoup
import pandas as pd
import openai
from graphdatascience import GraphDataScience

In [ ]:
api_key = <key>
NEO4J_URI = "neo4j://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = <pw>
import os
os.environ["OPENAI_API_KEY"] = api_key
openai.api_key = api_key

In [8]:
def get_embedding(text_to_embed):
    # Embed a line of text
    response = openai.Embedding.create(
        model= "text-embedding-ada-002",
        input=json.dumps([text_to_embed])
    )
    # Extract the AI output embedding as a list of floats
    embedding = response["data"][0]["embedding"]
    
    return embedding

In [28]:
import re
df = pd.read_excel('DORA_EN_Finale.xlsx')

def get_subarticles(article):
    return [(sub_art, get_embedding(sub_art)) for ix, sub_art in enumerate(re.split(r"\d.\xa0\xa0\xa0", article)) if sub_art != "" ]

[(1,
  'Financial entities shall have a sound, comprehensive and well-documented ICT risk management framework as part of their overall risk management system, which enables them to address ICT risk quickly, efficiently and comprehensively and to ensure a high level of digital operational resilience.'),
 (2,
  'The ICT risk management framework shall include at least strategies, policies, procedures, ICT protocols and tools that are necessary to duly and adequately protect all information assets and ICT assets, including computer software, hardware, servers, as well as to protect all relevant physical components and infrastructures, such as premises, data centres and sensitive designated areas, to ensure that all information assets and ICT assets are adequately protected from risks including damage and unauthorised access or usage.'),
 (3,
  'In accordance with their ICT risk management framework, financial entities shall minimise the impact of ICT risk by deploying appropriate strateg

In [44]:
sub_articles = []
for index, row in df.iterrows():
    for ix, sub, embedding in [(ix, sub_art, get_embedding(sub_art)) for ix, sub_art in enumerate(re.split(r"\d.\xa0\xa0\xa0", row["Textes de lois"])) if sub_art != "" ]:
        sub_article = {
            "from_article": row.Articles.replace("\xa0"," "),
            "title_article": row["énnoncer de l'article"],
            "sub_article_ix": ix,
            "sub_article_text": sub,
            "sub_article_text_embedding": embedding
        }
        sub_articles.append(sub_article)

In [45]:
df_sub_emb = pd.DataFrame.from_dict(sub_articles)

In [46]:
df_sub_emb.to_csv('dora_sub_articles_with_embedding.csv')

In [10]:
df['paragraph_embedding'] = df['Textes de lois'].map(get_embedding)

In [11]:
df.to_csv('dora_paragraphs_with_embedding.csv')